In [8]:

import gym
import numpy as np
import random
import matplotlib.pyplot as plt
import matplotlib
import warnings
warnings.filterwarnings("ignore")
import os
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from collections import deque
from config import *
custom_map = [
    'SFFFF',
    'FFFFF',
    'FFFFF',
    'FFFFF',
    'FFFFG'
]

s_size = 2

In [9]:
env = gym.make("FrozenLake-v1", desc=custom_map, is_slippery=False)
train_episodes=200
test_episodes=100
max_steps=300
state_size = env.observation_space.n
action_size = env.action_space.n
batch_size=32
print(state_size, action_size)

25 4


In [10]:
class Agent:
    def __init__(self, state_size, action_size):
        self.memory = deque(maxlen=2500)
        self.learning_rate=0.001
        self.epsilon=1
        self.max_eps=1
        self.min_eps=0.01
        self.eps_decay = 0.001/3
        self.gamma=0.9
        self.state_size= state_size
        self.action_size= action_size
        self.epsilon_lst=[]
        self.model = self.buildmodel()

    def buildmodel(self):
        model=Sequential()
        model.add(Dense(32, input_dim=self.state_size, activation='relu'))
        model.add(Dense(32, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse', optimizer=Adam(lr=self.learning_rate))
        return model

    def add_memory(self, new_state, reward, done, state, action):
        self.memory.append((new_state, reward, done, state, action))

    def action(self, state, episode, test = False):
        if np.random.rand() < self.epsilon and episode<=30 and not test:
            print("random action")
            return np.random.randint(0,4)
        return np.argmax(self.model.predict(state))

    def replay(self, batch_size, episode):
        minibatch=random.sample(self.memory, batch_size)
        for new_state, reward, done, state, action in minibatch:
            target= reward
            if not done:
                target=reward + self.gamma* np.amax(self.model.predict(new_state))
            target_f= self.model.predict(state)
            target_f[0][action]= target
            self.model.fit(state, target_f, epochs=1, verbose=0)

        if self.epsilon > self.min_eps:
            self.epsilon = MIN_EPSILON + (MAX_EPSILON - MIN_EPSILON) * \
            np.exp(-DECAY_RATE*episode)
        print("epsilon ", self.epsilon)
        self.epsilon_lst.append(self.epsilon)

    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)

In [11]:
# model2=Sequential()
# model2.add(Dense(32, input_dim=state_size, activation='relu'))
# model2.add(Dense(32, activation='relu'))
# model2.add(Dense(action_size, activation='linear'))
# model2.compile(loss='mse', optimizer=Adam(lr=0.001))
# model2.summary()
agent=Agent(state_size, action_size)


In [12]:
state_arr=np.zeros(state_size)
state_arr[23] = 1
state= np.reshape(state_arr, [1, state_size])

In [13]:
enemy_state_arr = np.zeros(state_size)
enemy_state_arr[24] = 1
enemy_state =  np.reshape(enemy_state_arr, [1, state_size])

In [14]:
reward_lst=[]
wins = 0
for episode in range(train_episodes):
    state= env.reset()
    state = np.random.choice(state_size-5)
    enemy_state = 24
    print("state is ", state)
    state_arr=np.zeros(state_size)
    state_arr[[state, enemy_state]] = 1
    state= np.reshape(state_arr, [1, state_size])
    reward = 0
    done = False
    for t in range(max_steps):
        # env.render()
        action = agent.action(state, episode)
        new_state, reward, done, info = env.step(action)
        new_state_arr = np.zeros(state_size)
        new_state_arr[[new_state, enemy_state]] = 1
        new_state = np.reshape(new_state_arr, [1, state_size])

        # next_state = new_state
        # old_state = state
        agent.add_memory(new_state, reward, done, state, action)
        state = new_state

        if done:
            wins+=1
            print(f'Episode: {episode:4}/{train_episodes} and step: {t:4}. Wins: {wins}/{episode:4}, reward {reward}')
            break

    # print("storing", next_state, reward, done, old_state, action)
    reward_lst.append(reward)

    if len(agent.memory)> batch_size:
        agent.replay(batch_size, episode)
print(' Train mean % score= ', round(100*np.mean(reward_lst),1))

state is  3
random action
random action
random action
random action
random action
random action
random action
random action
random action
random action
random action
random action
random action
random action
random action
random action
random action
random action
random action
random action
random action
random action
random action
random action
random action
random action
random action
random action
random action
Episode:    0/200 and step:   28. Wins: 1/   0, reward 1.0
state is  5
random action
random action
random action
random action
random action
random action
random action
random action
random action
random action
random action
random action
random action
random action
random action
random action
random action
random action
random action
random action
random action
random action
random action
random action
random action
random action
random action
random action
random action
random action
random action
random action
random action
random action
random action
random action
random 

In [15]:
agent.save("./saved_models/dqn_agent10x10.h5")
agent.load("./saved_models/dqn_agent10x10.h5")

In [16]:
# new_state_arr = np.zeros(25)
# new_state_arr[[24,24]] = 1
# new_state = np.reshape(new_state_arr, [1, 25])
# new_state

In [17]:
# test
test_wins=[]
for episode in range(test_episodes):
    state = env.reset()
    state = np.random.choice(state_size-5)
    enemy_state = 24
    print("state is ", state)
    state_arr=np.zeros(state_size)
    state_arr[[state, enemy_state]] = 1
    state= np.reshape(state_arr, [1, state_size])
    done = False
    reward=0
    state_lst = []
    state_lst.append(state)
    print('******* EPISODE ',episode, ' *******')

    for step in range(max_steps):
        action = agent.action(state, episode, test = True)
        new_state, reward, done, info = env.step(action)
        new_state_arr = np.zeros(state_size)
        new_state_arr[[new_state, enemy_state]] = 1
        new_state = np.reshape(new_state_arr, [1, state_size])
        state = new_state
        state_lst.append(state)
        if done:
            print(reward)
            # env.render()
            break

    test_wins.append(reward)
env.close()

print(' Test mean % score= ', int(100*np.mean(test_wins)))

state is  7
******* EPISODE  0  *******
1.0
state is  3
******* EPISODE  1  *******
1.0
state is  6
******* EPISODE  2  *******
1.0
state is  7
******* EPISODE  3  *******
1.0
state is  0
******* EPISODE  4  *******
1.0
state is  14
******* EPISODE  5  *******
1.0
state is  13
******* EPISODE  6  *******
1.0
state is  9
******* EPISODE  7  *******
1.0
state is  10
******* EPISODE  8  *******
1.0
state is  16
******* EPISODE  9  *******
1.0
state is  4
******* EPISODE  10  *******
1.0
state is  16
******* EPISODE  11  *******
1.0
state is  2
******* EPISODE  12  *******
1.0
state is  17
******* EPISODE  13  *******
1.0
state is  15
******* EPISODE  14  *******
1.0
state is  0
******* EPISODE  15  *******
1.0
state is  14
******* EPISODE  16  *******
1.0
state is  15
******* EPISODE  17  *******
1.0
state is  14
******* EPISODE  18  *******
1.0
state is  8
******* EPISODE  19  *******
1.0
state is  18
******* EPISODE  20  *******
1.0
state is  13
******* EPISODE  21  *******
1.0
state is

In [18]:
state = np.random.choice(state_size-5)
enemy_state = 24
state_arr=np.zeros(state_size)
state_arr[[state, enemy_state]] = 1
state= np.reshape(state_arr, [1, state_size])
state

array([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 1.]])

In [19]:
state = np.random.choice(state_size-5)
enemy_state = 24
print("state is ", state)
state_arr=np.zeros(state_size)
state_arr[[state, enemy_state]] = 1
state= np.reshape(state_arr, [1, state_size])
print(state)

state is  2
[[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  1.]]


In [20]:
new_state, reward, done, info = env.step(action)
print(new_state)
new_state_arr = np.zeros(state_size)
new_state_arr[[new_state, enemy_state]] = 1
new_state = np.reshape(new_state_arr, [1, state_size])
new_state

24


array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 1.]])